# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

C:\Users\USER\.conda\envs\machine learning\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# tf.__version__

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 8362 images belonging to 3 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (128, 128),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 2008 images belonging to 3 classes.


In [5]:
# from sklearn.preprocessing import MinMaxScaler
# SC = MinMaxScaler()
# training_set = SC.fit()

## Part 2 - Building the CNN

### Initialising the CNN

In [6]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[128, 128, 3]))

### Step 2 - Pooling

In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2) , strides=(2,2)))

### Adding a second convolutional layer

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2) , strides=(2,2)))

### Adding a 3rd convolutional layer

In [10]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2) , strides=(2,2)))

### Adding a 4th convolutional layer

In [11]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))

### Step 3 - Flattening

In [12]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [13]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [14]:
cnn.add(tf.keras.layers.Dense(units=3, activation='softmax'))

## Part 3 - Training the CNN

### Compiling the CNN

In [15]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [16]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 59, 59, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 29, 29, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 27, 27, 64)        36928     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 25, 25, 64)        3

### Training the CNN on the Training set and evaluating it on the Test set

In [17]:
#cnn.fit(x = training_set, validation_data = test_set, epochs = 25)
cnn.fit( training_set, validation_data= test_set, epochs=25)

Epoch 1/25
262/262 [==============================] - 195s 743ms/step - loss: 0.7489 - accuracy: 0.5088 - val_loss: 0.7341 - val_accuracy: 0.5319
Epoch 2/25
262/262 [==============================] - 192s 734ms/step - loss: 0.6919 - accuracy: 0.5674 - val_loss: 0.6950 - val_accuracy: 0.5573
Epoch 3/25
262/262 [==============================] - 204s 779ms/step - loss: 0.6587 - accuracy: 0.6178 - val_loss: 0.7151 - val_accuracy: 0.5837
Epoch 4/25
262/262 [==============================] - 214s 816ms/step - loss: 0.6420 - accuracy: 0.6384 - val_loss: 0.6172 - val_accuracy: 0.6868
Epoch 5/25
262/262 [==============================] - 238s 908ms/step - loss: 0.6003 - accuracy: 0.6807 - val_loss: 0.5693 - val_accuracy: 0.7176
Epoch 6/25
262/262 [==============================] - 219s 838ms/step - loss: 0.5640 - accuracy: 0.7069 - val_loss: 0.5942 - val_accuracy: 0.6982
Epoch 7/25
262/262 [==============================] - 242s 923ms/step - loss: 0.5385 - accuracy: 0.7344 - val_loss: 0.5638 -

In [19]:
# 模型結構存檔
from keras.models import model_from_json
json_string = cnn.to_json()
with open("cnn.config", "w") as text_file:
    text_file.write(json_string)

    
# 模型訓練結果存檔
cnn.save_weights("cnn.weight")

## Part 4 - Making a single prediction

In [31]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('./dataset/single_prediction/4.jpg', target_size = (128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
prediction = cnn.predict(test_image/255.0)
print(training_set.class_indices)

#当使用predict()方法进行预测时，返回值是数值，表示样本属于每一个类别的概率，我们可以使用numpy.argmax()方法找到样本以最大概率所属的类别作为样本的预测标签
prediction = np.argmax(prediction)
print(prediction)

{'apples': 0, 'cats': 1, 'dogs': 2}
2


In [ ]:
#心得:
#常見的binary classify 改成 multi classify, 但看起來 apples 這一項因為training set 樣本數不夠多, 導致分辨apple 的辨識度較差. 
#除了多增加樣本數外, 可能要改成用多物件辨識的方法才能提升辨識度